In [55]:
from pathlib import Path
from langchain_text_splitters import RecursiveCharacterTextSplitter
from src.infrastructure.loader import DocumentLoader
from langchain.embeddings import SentenceTransformerEmbeddings
import chromadb
from langchain_chroma import Chroma

In [53]:
embedder = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
chroma_client = chromadb.PersistentClient('./data/chroma')
chroma_client.get_or_create_collection("research_docs")

Collection(name=research_docs)

In [ ]:
vector_store = Chroma(
    embedding_function=embedder,
    client=chroma_client,
    collection_name="research_docs"
)
loader = DocumentLoader(recursive=True, glob=['*.md', '*.txt'], paths=[Path('./docs')])
splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=50,
    add_start_index=True,
)

StopIteration: 

In [ ]:



documents = loader.lazy_load()
for doc in documents:
    print(doc)


In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100, add_start_index=True)
chunks = splitter.split_documents(documents)

In [ ]:
for chunk in chunks:
    chunk.metadata['hash'] = gen_hash(chunk.page_content)

In [ ]:
db

In [ ]:
documents = db.similarity_search_with_relevance_scores("What is data science?", k=3)
[document for document in documents if document[1] > 0.65]

[(Document(id='c6991401-f36e-4a26-8b29-118ebb67f860', metadata={'source': './docs/data_science_overview.txt', 'start_index': 348, 'hash': 'd56174581a0ec6ce0a6111a80617024e'}, page_content='What is Data Science?\n\nData science is the practice of deriving actionable insights from data through a combination of:\n- Statistical analysis\n- Machine learning\n- Data visualization\n- Domain knowledge\n- Programming skills\n- Business acumen\n\nThe goal is to transform raw data into meaningful information that can drive decision-making and solve real-world problems.\n\nThe Data Science Process'),
  np.float32(0.7887581)),
 (Document(id='3c94c512-d26d-4d1f-8a84-f0cec53a3ae0', metadata={'source': './docs/data_science_overview.txt', 'start_index': 0, 'hash': '4292541e16c9140bd5390294860997bf'}, page_content='Data Science: A Comprehensive Overview\n\nData science is an interdisciplinary field that uses scientific methods, processes, algorithms, and systems to extract knowledge and insights from st